# Install, Paths and Parameters

In [1]:
# This extension reloads external Python files
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import math
from collections import defaultdict

import torch
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm
import random
import sys
from torch.utils.data import random_split
from matplotlib import pyplot as plt

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino
from src.model.train import *
from src.model.data import *

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')
# Path for intermediate outputs
BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc-fixed-labels/')
#BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc-subset/')

# Original Dataset
ORI_PATH = Path(DATA_PATH, 'ori_data/')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

TR_PATH = Path(ORI_PATH, 'train/')
TR_ORI_LABEL_PATH = Path(TR_PATH,'correct_labels.txt')
TR_ORI_IMAGES_PATH = Path(TR_PATH,'images')

VAL_PATH = Path(ORI_PATH, 'validation/')
VAL_ORI_LABEL_PATH = Path(VAL_PATH,'correct_labels.txt')
VAL_ORI_IMAGES_PATH = Path(VAL_PATH,'images')

# DAmageNet
DN_PATH = Path(DATA_PATH, 'damageNet')
DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
DN_IMAGES_PATH = Path(DN_PATH, 'images')
DN_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'damagenet')
DN_POSTHOC_LABEL_PATH = Path(DN_POSTHOC_PATH, 'labels.csv')

# PGD
TR_PGD_PATH = Path(MAX_PATH, 'adversarial_data/pgd_06/train')
TR_PGD_LABEL_PATH = TR_ORI_LABEL_PATH
TR_PGD_IMAGES_PATH = Path(TR_PGD_PATH, 'images')
TR_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/train/')
TR_PGD_POSTHOC_LABEL_PATH = Path(TR_PGD_POSTHOC_PATH, 'labels.csv')

VAL_PGD_PATH = Path(MAX_PATH, 'adversarial_data/pgd_06/validation')
VAL_PGD_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_PGD_IMAGES_PATH = Path(VAL_PGD_PATH, 'images')
VAL_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/validation/')
VAL_PGD_POSTHOC_LABEL_PATH = Path(VAL_PGD_POSTHOC_PATH, 'labels.csv')

# CW
TR_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/train')
TR_CW_LABEL_PATH = TR_ORI_LABEL_PATH
TR_CW_IMAGES_PATH = Path(TR_CW_PATH, 'images')
TR_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/train/')
TR_CW_POSTHOC_LABEL_PATH = Path(TR_CW_POSTHOC_PATH, 'labels.csv')

VAL_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/validation')
VAL_CW_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_CW_IMAGES_PATH = Path(VAL_CW_PATH, 'images')
VAL_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/validation/')
VAL_CW_POSTHOC_LABEL_PATH = Path(VAL_CW_POSTHOC_PATH, 'labels.csv')

# FGSM
TR_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/train')
TR_FGSM_LABEL_PATH = TR_ORI_LABEL_PATH
TR_FGSM_IMAGES_PATH = Path(TR_FGSM_PATH, 'images')
TR_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/train/')
TR_FGSM_POSTHOC_LABEL_PATH = Path(TR_FGSM_POSTHOC_PATH, 'labels.csv')

VAL_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/validation')
VAL_FGSM_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_FGSM_IMAGES_PATH = Path(VAL_FGSM_PATH, 'images')
VAL_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/validation/')
VAL_FGSM_POSTHOC_LABEL_PATH = Path(VAL_FGSM_POSTHOC_PATH, 'labels.csv')

In [2]:
# If CLASS_SUBSET is specified, INDEX_SUBSET will be ignored. Set CLASS_SUBSET=None if you want to use indexes.
# INDEX_SUBSET = get_random_indexes(number_of_images = 50000, n_samples=1000)
# CLASS_SUBSET = get_random_classes(number_of_classes = 25, min_rand_class = 1, max_rand_class = 1001)


CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)
# CLASS_SUBSET = CLASS_SUBSET[:2]

INDEX_SUBSET = None
NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 64

DEVICE = 'cuda'

In [3]:
datasets_paths = {
            'cw':{ 
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_CW_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_CW_IMAGES_PATH
                    }
                }
            },
            'ori':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_ORI_IMAGES_PATH
                    },
                    'val':{
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_ORI_IMAGES_PATH
                    }
                }
            },
            'dn':{
                'b':{
                    'train':{
                        'label':TR_CW_PATH,
                        'images':None
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':DN_IMAGES_PATH
                    }
                 }
            },
            'fgsm_06':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_FGSM_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_FGSM_IMAGES_PATH
                    }
                 }
            },
            'pgd_06':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_PGD_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_PGD_IMAGES_PATH
                    }
                }
            }
}

In [4]:
datasets = ['ori', 'cw', 'pgd_06', 'fgsm_06']
for ds in datasets:
    ds_dict = datasets_paths[ds]
    ds_dict['p'] = {
        'train': { 
            'images': Path(BASE_POSTHOC_PATH, ds, 'train', 'images'),
            'label': Path(BASE_POSTHOC_PATH, ds, 'train', 'labels.csv')
        },
        'val': { 
            'images': Path(BASE_POSTHOC_PATH, ds, 'val', 'images'),
            'label': Path(BASE_POSTHOC_PATH, ds, 'val', 'labels.csv')
        }
    }
    

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [ ]:
# model, linear_classifier = get_dino()

# Forward Pass

In [5]:
def posthoc_forward_pass(datasets, datasets_paths):
    logger_dict = {}
    for ds in datasets:
        ds_b = datasets_paths[ds]['b']
        ds_p = datasets_paths[ds]['p']
        logger_dict[ds] = {}
        transform = ONLY_NORMALIZE_TRANSFORM
        if ds == 'ori':
            transform = ORIGINAL_TRANSFORM
        for tv in ['train', 'val']:
            print(f'''images: {ds_b[tv]['images']}\nlabel: {ds_b[tv]['label']}\npred: {ds_p[tv]['label']}''')
            data_set = ImageDataset(ds_b[tv]['images'], ds_b[tv]['label'], transform, class_subset=CLASS_SUBSET)
            data_loader = DataLoader(data_set, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, shuffle=False)
            print(f'''{ds}: {tv} {len(data_set)}''')
            logger_dict[ds][tv] = validate_network(model, linear_classifier, data_loader, adversarial_attack=None, tensor_dir=ds_p[tv]['images'], path_predictions=ds_p[tv]['label'])
    return logger_dict

# Create Labels

In [6]:
from functools import reduce

def merge_frames(frames, on_what=['file', 'true_labels'], how='left'):
    return reduce(lambda left, right:pd.merge(left, right, on=on_what, how=how), frames)

def get_merged_labels(datasets=['ori', 'cw', 'pgd_06', 'fgsm_06'], datasets_types=['train', 'val'], datasets_paths=datasets_paths, save_path=BASE_POSTHOC_PATH, get_df_dict = False):
    df_data_types = {}
    df_data = {}
    for tv in datasets_types:
        df_data[tv] = {}
        for ds in datasets:
            ds_dict = datasets_paths[ds]
            df_data[tv][ds] = pd.read_csv(ds_dict['p'][tv]['label'])
            df_data[tv][ds].rename(columns = {'pred_labels': ds+'_pred'}, inplace = True)
            if ds != 'ori':
                df_data[tv][ds] = pd.merge(df_data[tv][ds], df_data[tv]['ori'], on=['file', 'true_labels'], how='left')
            df_data[tv][ds].to_csv(Path(BASE_POSTHOC_PATH, ds, tv, 'labels_merged.csv'), sep=",", index=None)
        df_data_types[tv] = merge_frames(df_data[tv].values())
        if save_path is not None:
            df_data_types[tv].to_csv(Path(save_path,tv+'.csv'), sep=",", index=None)
    if get_df_dict:
        return df_data_types, df_data

    return df_data_types

    

In [7]:
df_types, df_data = get_merged_labels(get_df_dict=True)

In [9]:
df_types['train']

,file,true_labels,ori_pred_x,cw_pred,ori_pred_y,pgd_06_pred,ori_pred_x,fgsm_06_pred,ori_pred_y
0,n01873310_42267.JPEG,4,4,4,4,12,4,18,4
1,n02326432_39907.JPEG,11,11,11,11,13,11,9,11
2,n01608432_6128.JPEG,0,0,0,0,7,0,0,0
3,n04447861_3923.JPEG,23,23,23,23,18,23,18,23
4,n02101388_21983.JPEG,8,8,8,8,23,8,23,8
...,...,...,...,...,...,...,...,...,...
32176,n03777754_11639.JPEG,19,19,19,19,20,19,20,19
32177,n02927161_43568.JPEG,16,16,16,16,18,16,18,16
32178,n02398521_22023.JPEG,12,12,12,12,13,12,4,12
32179,n03888257_34756.JPEG,21,21,21,21,18,21,18,21


In [11]:
df_data['train']['cw']

,file,true_labels,cw_pred,ori_pred
0,n01873310_42267.JPEG,4,4,4
1,n02326432_39907.JPEG,11,11,11
2,n01608432_6128.JPEG,0,0,0
3,n04447861_3923.JPEG,23,23,23
4,n02101388_21983.JPEG,8,8,8
...,...,...,...,...
32176,n03777754_11639.JPEG,19,19,19
32177,n02927161_43568.JPEG,16,16,16
32178,n02398521_22023.JPEG,12,12,12
32179,n03888257_34756.JPEG,21,21,21
